## Model Selection and Modeling

- [Linear Regression: OLS](#lm)
- [Random Forest Regression](#rfr)
- [Light Gradient Boosting Model With Bayesian Optimization](#lgbm)

### Import Necessary Modules and Datasets

In [1]:
# Import tools to get datasets
import urllib
import requests
import io

# Import modules for data reading and plots
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Import all for model selections and modelling
from bayes_opt import BayesianOptimization
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
import time

In [2]:
start = time.time() # Timer starts

Xtrainurl = 'https://raw.githubusercontent.com/jonahwinninghoff/Springboard_Capstone_Project/main/Assets/X_train'
Xvalidurl = 'https://raw.githubusercontent.com/jonahwinninghoff/Springboard_Capstone_Project/main/Assets/X_valid'
Xtestiurl = 'https://raw.githubusercontent.com/jonahwinninghoff/Springboard_Capstone_Project/main/Assets/X_test'

ytrainurl = 'https://github.com/jonahwinninghoff/Springboard_Capstone_Project/blob/main/Assets/y_train?raw=true'
yvalidurl = 'https://github.com/jonahwinninghoff/Springboard_Capstone_Project/blob/main/Assets/y_valid?raw=true'
ytestiurl = 'https://github.com/jonahwinninghoff/Springboard_Capstone_Project/blob/main/Assets/y_test?raw=true'

In [3]:
# Read X_train dataset
url = urllib.request.urlopen(Xtrainurl)
file = io.BytesIO(url.read())
X_train = pd.read_csv(file, encoding='cp1252').drop('Unnamed: 0', 
                                                    axis=1)

# Read X_valid dataset
url = urllib.request.urlopen(Xvalidurl)
file = io.BytesIO(url.read())
X_valid = pd.read_csv(file, encoding='cp1252').drop('Unnamed: 0', 
                                                    axis=1)

# Read X_test dataset
url = urllib.request.urlopen(Xtestiurl)
file = io.BytesIO(url.read())
X_test = pd.read_csv(file, encoding='cp1252').drop('Unnamed: 0', 
                                                    axis=1)

In [4]:
# Read y_train dataset
response = requests.get(ytrainurl)
response.raise_for_status()
y_train = np.load(io.BytesIO(response.content))

# Read y_valid dataset
response = requests.get(yvalidurl)
response.raise_for_status()
y_valid = np.load(io.BytesIO(response.content))

# Read y_test dataset
response = requests.get(ytestiurl)
response.raise_for_status()
y_test = np.load(io.BytesIO(response.content))

### Linear Regression: OLS <a id ='lm'></a>

In [5]:
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train)
fitted_lm = model.fit()

display(fitted_lm.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.696
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                 6.464e+04
Date:                Fri, 03 Sep 2021   Prob (F-statistic):               0.00
Time:                        11:08:05   Log-Likelihood:                 24551.
No. Observations:              141159   AIC:                        -4.909e+04
Df Residuals:                  141153   BIC:                        -4.903e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.1596      0.007     23.924      0.000       0.146       0.173
cosine median     -0.0452      0.002    -19.139      0.000      -0.050      -0.041
cosine mean        0.7183      0.015     47.088      0.000       0.688       0.748
cosine minimum    -0.1735      0.009    -20.246      0.000      -0.190      -0.157
cosine maximum     0.6937      0.003    198.503      0.000       0.687       0.701
cosine std        -0.6243      0.035    -18.031      0.000      -0.692      -0.556
==============================================================================
Omnibus:                    16213.742   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            28286.985
Skew:                           0.786   Prob(JB):                         0.00
Kurtosis:                       4.530   Cond. No.                         80.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Each variable is statistically significant at two-sided 1% alpha level. The F-statistics shows that at least one variable has a significant impact on target variable. The R-squared score is 69.6%. The adjusted R-squared score shows that increase in number of parameters has no effect on R-squared score. Akaike and Bayesian Information Criterion are -9.344 and −9.327, respectively. This needs to evaluate by comparing TFIDF without cosine similarities. In general sense, the lower the score is, the better this model is. The skewness is 0.789, which is not a problem. The kurtosis score shows that this distribution is leptokurtotic. 

In [6]:
X_valid = sm.add_constant(X_valid)
y_predicted = fitted_lm.predict(X_valid)

In [7]:
def r2_calculator(predict, true):
    merged = pd.concat([pd.DataFrame(list(predict)).rename(columns={0:'predicted'}), 
           pd.DataFrame(list(true)).rename(columns={0:'true'})],axis=1)
    
    # 1 - (predict - true)^2/(true - mean(true))^2 = 1 - RSS/TSS = r_2
    r2 = 1 - sum((merged['predicted'] - merged['true'])**2)/sum((merged['true'] - np.mean(merged['true']))**2)
    
    return r2

def adjusted_r2_calculator(predict, true):
    r2 = r2_calculator(predict, true) # r_2
    n = len(true)                     # number of rows
    k = len(fitted_lm.params)         # number of parameters
    
    adj_r2 = 1 - ((1-r2)*(n-1))/(n-k-1)    # 1 - [(1 - r_2)*(n - 1)/(n - k - 1)] = adjusted r^2
    
    return adj_r2

def mae_and_mse_calculator(predict, true):
    merged = pd.concat([pd.DataFrame(list(predict)).rename(columns={0:'predicted'}), 
           pd.DataFrame(list(true)).rename(columns={0:'true'})],axis=1)
    mae = sum(np.abs(merged['true'] - merged['predicted']))/len(true) # 1/n ∑ |true - predict|   = MAE
    mse = sum((merged['true'] - merged['predicted'])**2)/len(true)    # 1/n ∑ (true - predict)^2 = MSE
    
    return mae, mse

In [8]:
print('Validation Set')
print('R2: '+str(round(r2_calculator(y_predicted, y_valid),4)))
print('Adjusted R2: '+str(round(adjusted_r2_calculator(y_predicted,y_valid),4)))
print('MAE: '+ str(round(mae_and_mse_calculator(y_predicted,y_valid)[0],4)))
print('RMSE: '+ str(round(mae_and_mse_calculator(y_predicted,y_valid)[1]**0.5,4)))

Validation Set
R2: 0.6928
Adjusted R2: 0.6928
MAE: 0.1531
RMSE: 0.2036


When the model is being generalized using the validation set, the R2 score decreases from 69.6% to 69.28%. The RMSE score shows that if the target variable is 50%, then it can be either 70.36% and 29.64%. The MAE score means that if it is 50%, it can be either 65.13% and 34.87%. Non-differentiable although the MAE is, it is resistant to outliers. 

## Random Forest Regression <a id='rfr'></a>

In [9]:
rfr_model = RandomForestRegressor()
rfr_model.fit(X_train, y_train.ravel())

RandomForestRegressor()

In [10]:
print('Validation Set')
print('R2: ' + str(round(r2_calculator(rfr_model.predict(X_valid),y_valid),4)))
print('Adjusted R2: ' + str(round(adjusted_r2_calculator(rfr_model.predict(X_valid),y_valid),4)))
print('MAE: ' + str(round(mae_and_mse_calculator(rfr_model.predict(X_valid),y_valid)[0],4)))
print('RMSE: ' + str(round(mae_and_mse_calculator(rfr_model.predict(X_valid),y_valid)[1]**0.5,4)))

Validation Set
R2: 0.9091
Adjusted R2: 0.9091
MAE: 0.0687
RMSE: 0.1107


As a result, the R2 and Adjusted R2 are increased. The MAE and RMSE are decreased. In other words, the random forest model outperforms the OLS model.

## Light Gradient Boosting Model With Bayesian Optimization <a id='lgbm'></a>

In [11]:
def lgbm_eval(lambda_l2,lambda_l1,max_depth,learning_rate,n_estimators):
    lgbm_model = LGBMRegressor(lambda_l2 = lambda_l2, lambda_l1 = lambda_l1, 
                               max_depth = int(round(max_depth,0)),
                               learning_rate = learning_rate, 
                               n_estimators = int(round(n_estimators,0)))
    
    lgbm_model.fit(X_train, y_train.ravel())
    return r2_calculator(lgbm_model.predict(X_valid),y_valid)

In [12]:

lgbmBO = BayesianOptimization(lgbm_eval, {'lambda_l2':(0, 0.5),
                                          'lambda_l1':(0,0.5),
                                          'max_depth':(-1,6),
                                          'learning_rate':(0.01,0.5),
                                          'n_estimators':(10,200)})

lgbmBO.maximize(n_iter=20, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------------------
|  1        |  0.9091   |  0.1413   |  0.1312   |  0.3061   |  3.908    |  175.7    |
|  2        |  0.9091   |  0.3944   |  0.2276   |  0.3327   |  4.729    |  137.5    |
|  3        |  0.9091   |  0.4067   |  0.4972   |  0.07254  |  5.872    |  137.1    |
|  4        |  0.9091   |  0.1751   |  0.3837   |  0.4774   |  5.778    |  136.2    |
|  5        |  0.9091   |  0.4002   |  0.3125   |  0.183    |  4.855    |  137.5    |
|  6        |  0.9091   |  0.4001   |  0.2491   |  0.2314   |  5.922    |  137.4    |
|  7        |  0.9091   |  0.2171   |  0.4457   |  0.3516   |  5.861    |  137.1    |
|  8        |  0.9091   |  0.2702   |  0.4877   |  0.1242   |  5.826    |  136.8    |
|  9        |  0.9091   |  0.2792   |  0.2543   |  0.4018   |  4.794    |  137.4    |
|  10       |  0.9083   |  0.2033   |  0.3061   |  0.3

In [13]:
print(lgbmBO.max['params'])

{'lambda_l1': 0.4066706373447378, 'lambda_l2': 0.4972157734835887, 'learning_rate': 0.0725439481936864, 'max_depth': 5.8722982849661, 'n_estimators': 137.05887878322778}


In [14]:
lgbm_model = LGBMRegressor(lambda_l2 = lgbmBO.max['params']['lambda_l1'],
                           lambda_l1 = lgbmBO.max['params']['lambda_l2'],
                           max_depth = int(round(lgbmBO.max['params']['max_depth'],0)),
                           learning_rate = lgbmBO.max['params']['learning_rate'],
                           n_estimators = int(round(lgbmBO.max['params']['n_estimators'],0)))
lgbm_model.fit(X_train, y_train.ravel())

LGBMRegressor(lambda_l1=0.4972157734835887, lambda_l2=0.4066706373447378,
              learning_rate=0.0725439481936864, max_depth=6, n_estimators=137)

In [15]:
print('Validation Set')
print('R2: ' + str(round(r2_calculator(lgbm_model.predict(X_valid),y_valid),4)))
print('Adjusted R2: ' + str(round(adjusted_r2_calculator(lgbm_model.predict(X_valid),y_valid),4)))
print('MAE: ' + str(round(mae_and_mse_calculator(lgbm_model.predict(X_valid),y_valid)[0],4)))
print('RMSE: ' + str(round(mae_and_mse_calculator(lgbm_model.predict(X_valid),y_valid)[1]**0.5,4)))

Validation Set
R2: 0.9091
Adjusted R2: 0.9091
MAE: 0.0687
RMSE: 0.1107


When the hyperparameter search completes, the result indicates that the error is irreducible. Two ways to reduce this error are feature engineering and data wrangling

In [16]:
print(f'Time taken to run: {time.time() - start} seconds')

Time taken to run: 32.09386587142944 seconds
